In [1]:
import os
import json
import yaml

import dspy

/Users/dkhundley/Documents/Repositories/genai-misc/dspy_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../keys/api_keys.yaml') as f:
    api_keys = yaml.safe_load(f)

In [3]:
model = dspy.OpenAI(model = 'gpt-4o-mini', api_key = api_keys['OPENAI_API_KEY'])
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
# dspy.settings.configure(lm = model)
dspy.settings.configure(lm = model, rm = colbertv2_wiki17_abstracts)

In [4]:
from dspy.datasets import HotPotQA

dataset = HotPotQA(train_seed = 1, train_size = 20, eval_seed = 2023, dev_size = 50, test_size = 0)

trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

In [5]:
trainset

[Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'}),
 Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'}),
 Example({'question': 'Which of these publications was most recently published, Who Put the Bomp or Self?', 'answer': 'Self'}) (input_keys={'question'}),
 Example({'question': 'The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?', 'answer': '1950'}) (input_keys={'question'}),
 Example({'question': 'Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?', 'answer': 'Tae Kwon Do Times'}) (input_keys={'question'}),
 Example({'question': 'In what year was the club founded that played Manchester City in the 1972 FA Charity Shield', 'answer': '1874'}) (input_keys={'question'}),
 Example({'question': 'Which is taller, the E

In [6]:
class BasicQA(dspy.Signature):
    '''Answer questions with short factoid answers.'''

    question = dspy.InputField()
    answer = dspy.OutputField(desc = 'Often between 1 and 5 words')

In [7]:
# Defining the predictor
generate_answer = dspy.Predict(BasicQA)

In [8]:
dev_example = devset[18]
print(f"Question: {dev_example.question}")
print(f"Answer: {dev_example.answer}")
print(f"Relevant Wikipedia Titles: {dev_example.gold_titles}")

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Answer: English
Relevant Wikipedia Titles: {'Robert Irvine', 'Restaurant: Impossible'}


In [9]:
pred = generate_answer(question = dev_example.question)

 		You are using the client GPT3, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


In [10]:
pred

Prediction(
    answer='American'
)

In [11]:
model.inspect_history()




Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: Often between 1 and 5 words

---

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Answer: American





'\n\n\nAnswer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: Often between 1 and 5 words\n\n---\n\nQuestion: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?\nAnswer:\x1b American\x1b\n\n\n'